In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import constants as C
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
from astropy import units as u
import tqdm 

### 17th Feb:

**Conditional about observations:**

The user gives the latitude and the functions returns if the phenomenon can be see it or not. This using the conditional "altitude > 0 degrees" but with the difference of taking into account the time of observation on the altitude-Azimuth coordinate system; this is the equivalent ofr the conditional of "dec > 90-latitude".   

Alert system's input format: RA, Dec (hmsdms); Latitude, longitude (DMS); militar hour (hms) using J200 
(Reference: https://ztf.uw.edu/alerts/public/)

**About the sing on the coordinates**
 A positive value for North and East, a negative value for South and West. (Reference: https://tnp.uservoice.com/knowledgebase/articles/172110-latitude-longitude-formats-and-conversion)

In [9]:
#Playtime data
lat_obs = '4-35-56-N' 
lon_obs = '74-04-51-W'   
name = ['Aldebaran','Polar Star', 'Antares']
ra_list = ['04h35m55.64s','02h31m19.79s','16h29m24.17s'] 
dec_list = ['+16d30m27.2s','+89d16m10.10s','-26d25m53.56s']
date = '2025-02-17 21:30:00'

#moving time
date_i = '2025-02-17 19:30:00'
date_f = '2025-02-18 07:00:00'
timescale = ['m', 10]

In [10]:
#Convert the DMS format to degrees
def ConvertLaLo(l):
    l_ = l.split('-')
    D = float(l_[0])
    M = float(l_[1])
    S = float(l_[2])
    dir = l_[3]
    r = D + (M/60) + (S/3600)
    if dir=='W' or dir=='S':
        return r*(-1)
    else:
        return r

ConvertLaLo(lon_obs)

-74.08083333333333

In [ ]:
def Observations(longitude, latitude, RA, DEC, Date):
    time_obs = Time(Date)
    lat_conv = ConvertLaLo(latitude)
    lon_conv = ConvertLaLo(longitude)
    observer = EarthLocation(lat=lat_conv*u.deg, lon=lon_conv*u.deg)
    Conditionals = []

    for i in range(0,len(RA)):

        celestial_coord = SkyCoord(ra=RA[i], dec=DEC[i]) #mantain the degrees units

        # Calculate the coordenates AltAz for the time and observer
        frame_altaz = AltAz(obstime=time_obs, location=observer)
        altaz_coord = celestial_coord.transform_to(frame_altaz) #Transform for the J200 coordinate system for altaz
    
        # Determinate if its observable (altitude > 0 degrees), return a boolean
        Conditionals.append(altaz_coord.alt > 0*u.deg)

    #Put everything on a dataframe in the format for better reading
    Data = pd.DataFrame([])
    Data.style.set_caption(Date)
    Data['Observable'] = Conditionals
    Data['RA'] = RA
    Data['Dec'] = DEC

    return Data

Observations(lon_obs, lat_obs, ra_list, dec_list, date)

,Observable,RA,Dec
0,True,04h35m55.64s,+16d30m27.2s
1,True,02h31m19.79s,+89d16m10.10s
2,False,16h29m24.17s,-26d25m53.56s


### 21th Feb

Now we need to expand the function for a time slot, establish by the user. With this condition, is necesary to report if its observable. An idea is refreshing the dataframe with a nonsmall step, such we can determine if the function is returning what we want. 

Change the scale of time to UTC and keep the format iso for maintain the international format.

Actually, the function Time of astropy simplifies the code because allows sum seconds, hour and minutes in a line.
(Reference: https://docs.astropy.org/en/stable/api/astropy.time.Time.html#astropy.time.Time.FORMATS)

In [12]:
def MoveTime(time_actual,scale,sc):
    if scale == 's':
        t = time_actual + sc*u.second 
    if scale == 'm':
        t = time_actual + sc*u.minute
    if scale == 'h':
        t = time_actual + sc*u.hour

    return t

MoveTime(Time(date,scale='utc',format='iso'),'h',1)

<Time object: scale='utc' format='iso' value=2025-02-17 22:30:00.000>

In [13]:
#With time slot and optimized

def Observations(longitude, latitude, RA, DEC, Date_i, Date_f,time_scale):
    actual_time = Time(Date_i,format = 'iso', scale='utc')
    final_time = Time(Date_f,format = 'iso', scale='utc')
    lat_conv = ConvertLaLo(latitude)
    lon_conv = ConvertLaLo(longitude)
    observer = EarthLocation(lat=lat_conv*u.deg, lon=lon_conv*u.deg)
    Big_Data = []

    while actual_time <= final_time:
        Conditionals = []

        for i in range(0,len(RA)):

            celestial_coord = SkyCoord(ra=RA[i], dec=DEC[i]) #mantain the degrees units

            # Calculate the coordenates AltAz for the time and observer
            frame_altaz = AltAz(obstime=actual_time, location=observer)
            altaz_coord = celestial_coord.transform_to(frame_altaz) #Transform for the J200 coordinate system for altaz
    
            # Determinate if its observable (altitude > 0 degrees), return a boolean
            Conditionals.append(altaz_coord.alt > 0*u.deg)

        #Put everything on a dataframe in the format for better reading
        Data = pd.DataFrame([])
        Data['Observable'] = Conditionals
        Data['RA'] = RA
        Data['Dec'] = DEC
        Big_Data.append(Data) 

        actual_time = MoveTime(actual_time,time_scale[0],time_scale[1])

    return pd.concat(Big_Data, axis=1) 

Observations(lon_obs, lat_obs, ra_list, dec_list, date_i, date_f,timescale)

,Observable,RA,Dec,Observable,RA,Dec,Observable,RA,Dec,Observable,...,Dec,Observable,RA,Dec,Observable,RA,Dec,Observable,RA,Dec
0,True,04h35m55.64s,+16d30m27.2s,True,04h35m55.64s,+16d30m27.2s,True,04h35m55.64s,+16d30m27.2s,True,...,+16d30m27.2s,False,04h35m55.64s,+16d30m27.2s,False,04h35m55.64s,+16d30m27.2s,False,04h35m55.64s,+16d30m27.2s
1,True,02h31m19.79s,+89d16m10.10s,True,02h31m19.79s,+89d16m10.10s,True,02h31m19.79s,+89d16m10.10s,True,...,+89d16m10.10s,True,02h31m19.79s,+89d16m10.10s,True,02h31m19.79s,+89d16m10.10s,True,02h31m19.79s,+89d16m10.10s
2,False,16h29m24.17s,-26d25m53.56s,False,16h29m24.17s,-26d25m53.56s,False,16h29m24.17s,-26d25m53.56s,False,...,-26d25m53.56s,True,16h29m24.17s,-26d25m53.56s,True,16h29m24.17s,-26d25m53.56s,True,16h29m24.17s,-26d25m53.56s


### 22 Feb
Try to simulate for see in the time slot only the observables with the observer as the center. For the simulation is necessary to create a class. 

In [ ]:
# class Transients:
    
#     def __init__(longitude, latitude, RA, DEC, Date_i, Date_f,time_scale):
        
#         self.location = EarthLocation(lat=ConvertLaLo(latitude)*u.deg, lon=ConvertLaLo(longitude)*u.deg)
#         self.dt = time_scale[1] # Paso del tiempo
#         self.t_scale = time_scale[0] # Formato del tiempo        
#         self.date_i = Time(Date_i,format = 'iso', scale='utc') 
#         self.date_f = Time(Date_f,format = 'iso', scale='utc') 
        
#         self.G = 4*np.pi**2 # Unidades gaussianas
        
#         self.r = np.zeros(3)
#         self.v = np.zeros_like(self.r)
#         self.a = np.zeros_like(self.r)
        
#         self.r[0] = self.a_*(1-self.e)
#         self.v[1] = np.sqrt( self.G*(1+self.e)/(self.a_*(1.-self.e)) )
        
#         self.R = np.zeros((len(t),len(self.r)))
#         self.V = np.zeros_like(self.R)
        
#         # El valor del pasado
#         self.rp = self.r
        
#     def GetAceleration(self):
        
#         d = np.linalg.norm(self.r)
#         self.a = -self.G/d**3*self.r
        
        
#     def Evolution(self,i):
        
#         self.SetPosition(i)
#         self.SetVelocity(i)
#         self.GetAceleration()
        
#         if i==0:
#             self.r = self.rp + self.v*self.dt
#         else:
            
#             # rp pasado, r presente rf futuro
#             self.rf = 2*self.r - self.rp + self.a*self.dt**2
#             self.v = (self.rf - self.rp)/(2*self.dt)
            
#             self.rp = self.r
#             self.r = self.rf
    
#     def SetPosition(self,i):
#         self.R[i] = self.r
        
#     def SetVelocity(self,i):
#         self.V[i] = self.v
    
#     def GetPosition(self,scale=1):
#         return self.R[::scale]
    
#     def GetVelocity(self,scale=1):
#         return self.V[::scale]
    
#     def GetPerihelio(self):
        
#         Dist = np.linalg.norm(self.R,axis=1)
        
#         timeup = []
        
#         for i in range(1,len(Dist)-1):
#             if Dist[i] < Dist[i-1] and Dist[i] < Dist[i+1]:
#                 timeup.append(self.t[i])
            
#         return timeup

In [ ]:
# def GetPlanetas(t):
    
#     Mercurio = Planeta(0.2056,0.307,t)
#     Venus = Planeta(0.0067,0.7233,t)
#     Tierra = Planeta(0.01671,1.,t)
    
#     return [Mercurio,Venus,Tierra]

#This is the time scale
# dt = 0.001
# tmax = 5
# t = np.arange(0.,tmax,dt)
# Planetas = GetPlanetas(t)

In [ ]:
# def RunSimulation(t,Planetas):
    
#     for it in tqdm(range(len(t)), desc='Running simulation', unit=' Steps' ):
        
#         #print(it)
#         for i in range(len(Planetas)):
#             Planetas[i].Evolution(it)
#             # Aca debes agregar la interaccion con la pared
            
            
#     return Planetas

# Planetas = RunSimulation(t,Planetas)

In [ ]:
# fig = plt.figure(figsize=(8,5))
# ax = fig.add_subplot(221,projection='3d')
# ax1 = fig.add_subplot(222)
# ax2 = fig.add_subplot(223)

# colors=['r','k','b']

# def init():
    
#     ax.clear()
#     ax.set_xlim(-1,1)
#     ax.set_ylim(-1,1)
#     ax.set_zlim(-1,1)
    
#     ax1.clear()
#     ax1.set_xlim(-1,1)
#     ax1.set_ylim(-1,1) 
    
#     ax2.clear()
#     ax2.set_xlim(-2,2)
#     ax2.set_ylim(-2,2) 
    
# def Update(i):
    
#     init()
    
#     for j, p in enumerate(Planetas):
        
#         x = p.GetPosition(scale)[i,0]
#         y = p.GetPosition(scale)[i,1]
#         z = p.GetPosition(scale)[i,2]
        
#         vx = p.GetVelocity(scale)[i,0]
#         vy = p.GetVelocity(scale)[i,1]
#         vz = p.GetVelocity(scale)[i,2]
    
#         ax.scatter(0,0,0,s=200,color='y')
#         ax.quiver(x,y,z,vx,vy,vz,color=colors[j],length=0.03)
        
#         ax.scatter(x,y,z,color=colors[j])
        
#         circle = plt.Circle((x,y),0.1,color=colors[j],fill=True)
#         ax1.add_patch(circle)
    
#     # Mercurio visto desde tierra
#     Mx = Planetas[0].GetPosition(scale)[:i,0] - Planetas[2].GetPosition(scale)[:i,0]
#     My = Planetas[0].GetPosition(scale)[:i,1] - Planetas[2].GetPosition(scale)[:i,1]
    
#     # Venus visto desde tierra
#     Vx = Planetas[1].GetPosition(scale)[:i,0] - Planetas[2].GetPosition(scale)[:i,0]
#     Vy = Planetas[1].GetPosition(scale)[:i,1] - Planetas[2].GetPosition(scale)[:i,1]
    
#     ax2.scatter(Mx,My,marker='.',label='Mercurio')
#     ax2.scatter(Vx,Vy,marker='.',label='Venus')
    
# Animation = anim.FuncAnimation(fig,Update,frames=len(t1),init_func=init)